Jupyter Notebook: Visualize Fragility

Prepared by: Fatemeh Jalayer, Hossein Ebrahimian

Uses data from: Empirical Tsunami Risk Products Dataset (ETRiS v0, https://eurotsunamirisk.org/)

This Notebook describes how the fragility data available in EPOS-ICS-C portal under the layer entitled "Empirical Tsunami Risk Products Dataset (ETRiS v0)" can be visualized and interpreted.

The dataset provides risk products derived from observed data collected during past tsunami events. These risk products include tsunami fragility and vulnerability curves, which are essential components for tsunami risk assessment. The underlying observed data, which are not directly visualized in this layer, contain information on tsunami effects (e.g., tsunami height, flow depth) and tsunami consequences (e.g., casualties, building damage).
For each location, depending on the type of data available, either fragility or vulnerability curves can be visualized. 

This Jupyter Notebook focuses specifically on the visualization of fragility information.

A fragility curve represents the probability of exceeding a given damage level as a function of a tsunami intensity measure (e.g., flow depth) for a particular building class, or more generally, a specific asset at risk. 

In summary, defining a fragility curve requires three fundamental elements:

1- Damage scale:
The damage scale typically consists of a set of discrete, mutually exclusive, and collectively exhaustive damage states. In simple terms, an asset cannot be assigned two distinct damage states, and the defined states together cover all possible levels of damage.
For example, a possible damage scale may include: no damage, light damage, moderate damage, severe damage, collapse, completely washed away.
The damage states correspond to the intervals between these levels.

2- Taxonomy or class of asset at risk:
A taxonomy schema (e.g., GED4ALL) defines classes of exposed assets identified by taxonomy strings that describe the specific attributes of each class.
For example, the GED4ALL taxonomy string "MUR/HEX:2/YPRE:1939/RES" refers to two-storey residential unreinforced masonry buildings constructed before 1939 (adapted from the TSU vocabulary).

3- Fragility model:
A fragility model represents a probability distribution, specifically, a cumulative distribution function (CDF).
By specifying the underlying probability distribution (e.g., normal, lognormal, logistic, Gumbel, etc.), the functional form of the fragility curve is fully defined.

The procedure for visualization a fragility curve in ETRiS (v0) layer can be summarized as follows. 


1- Reading the input from CSV file on GitHub.
The input fragility data can be accessed directly from the GitHub repository:
https://github.com/eurotsunamirisk/etris_data_and_data_products/tree/main/etris_data_products/Fragility_Curves

As an example, consider “Wood buildings, 3 storeys or more” (W/HAPP:3, according to the GED4ALL taxonomy schema) in Japan, for Model 3. The corresponding CSV file is available at:
 https://github.com/eurotsunamirisk/etris_data_and_data_products/blob/main/etris_data_products/Fragility_Curves/Japan%202011_Wood%2C%203%20storey%20and%20more_M3.csv
    
(note that Three fragility models are available: Model 1, Model 2, and Model 3.)

You can repeat this process for all three fragility models by simply changing the model number in the file name.
Each fragility model corresponds to a different link function, as described below:

Model 1- Logistic link function: (1+exp(ax+b))^-1

Model 2- Probit link function: F(ax+b), where F is the standard Gaussian CDF. 

Model 3- cloglog link function: 1-exp(-exp(ax+b))

For detailed information on these fragility models, please see the following article:

Jalayer, F., Ebrahimian, H., Trevlopoulos, K., Bradley, B. (2023). Empirical tsunami fragility modelling for hierarchical damage levels. Natural Hazards and Earth System Sciences, 23(2), 909-931. https://nhess.copernicus.org/articles/23/909/2023/


Note: Before running the Python code, the following libraries should be installed:
pandas, plotly, numpy, scipy, math.
You can install all required dependencies using the requirements.txt file:

pip install -r requirements.txt

In [ ]:
# Initial Libraries

import pandas as pd
import plotly.graph_objects as ply
import numpy as np
import scipy.stats as ss
import math

Since the fragility data can be attributed to different damage scales and can have different number fo damaeg levels, we first detect the number of damage levels from the csv file and assign color codes. In general, the color codes vary from light green to red as the level of damage moves from no damage to complete desctruction.

In [ ]:
# Read CSV file

allData = pd.read_csv('Japan 2011_Wood, 3 storey and more_M3.csv',skiprows=[0])
intensityType = allData.columns[0];
IM = allData[intensityType]
size=np.shape(allData)             # read the size of the csv file
nCol=size[1]                       # read number of columns in the csv file
nDL=nCol//7                        # calculate the number of damage levels for which fragility is reported
print('Number of damage levels= ', nDL)
myColor= []                        
myColor.append("rgb(124,252,0)")   # color lawngreen
myColor.append("rgb(69,139,116)")  # color dark green 
myColor.append("rgb(255,215,0)")   # color yellow
myColor.append("rgb(255,97,3)")    # color orange
myColor.append("rgb(240,128,128)") # color coral
myColor.append("rgb(255,48,48)")   # color firebrick

# Show the CSV file
allData

(2) Plot the fragility curves: Plot flow depth versus mean, mean minus 1 sigma, mean plus 1 sigma fragility values for all damage levels detected. This step can be repeated for as as many damage levels for which the fragility information is available.

In [ ]:
# Plot the fragility curve for the nDL damage levels

fig = ply.Figure()
i=0
while i<nDL: 
    i +=1
    colMinus=7*(i-1)+1
    RF_minus1sigma = allData.iloc[:,colMinus]
    colMean=7*(i-1)+2
    RF = allData.iloc[:,colMean]
    colPlus=7*(i-1)+3
    RF_plus1sigma = allData.iloc[:,colPlus]
    fig.add_scatter(x=IM,y=RF,mode='lines',line=dict(color=myColor[i-1],width=3),name=('Mean fragility D'+str(i)))
    fig.add_scatter(x=IM,y=RF_minus1sigma,mode='lines',line=dict(color=myColor[i-1],width=0.5),opacity=0.5,showlegend=False)
    fig.add_scatter(x=IM,y=RF_plus1sigma,mode='lines',line=dict(color=myColor[i-1],width=0.5),opacity=0.5,fill='tonexty',name=('±1\u03C3 confidence interval D'+str(i)))

    fig.update_xaxes(showline=True,linecolor='black',title=intensityType, tickmode = 'linear',tick0 = 0.0,dtick = 0.5,ticklen=3, tickcolor='black',ticks="inside",showgrid=True)
    fig.update_yaxes(showline=True,linecolor='black',title='Probability of exceeding damage level', tickmode = 'linear',tick0 = 0.0,dtick = 0.05,ticklen=3, tickcolor='black',ticks="inside",showgrid=True)
    
    #bgColor = 'lightgray'
    bgColor = 'rgb(189,189,189)'
    
    fig.update_layout(width=1000,height=800,plot_bgcolor=bgColor,legend=dict(bordercolor='black',borderwidth=0.5,x=1.0,y=0.0,xanchor='right',yanchor='bottom'),title =dict(text='Fragility and its confidence interval',x=0.075,font=dict(size=30)),)

    fig.update_xaxes(rangemode="tozero")
    fig.update_yaxes(range=[0,1.02])

    fig.update_xaxes(showspikes=True)
    fig.update_yaxes(showspikes=True)

fig.show()

(3) Estimate the fragility parameters for a chosen damage level. In this step, the median and the logarithmic standard deviation of the fragility curve for a chosen damage level smaller than or equal to nDL can be estimated. The median shows the flow depth with 50% probability of being exceeded. The logarithmic standard deviation is equal to half the ratio between the 84th and 16th percentile intensity measures and is a measure of the spread of the fragility curve.

In [ ]:
# Estimating the parameters of the fragility curve

i=5      # Enter here the damage state for which to plot the statistics

print('The damage level for which to view statistics= ', i)

colMinus=7*(i-1)+1
RF_minus1sigma = allData.iloc[:,colMinus]
colMean=7*(i-1)+2
RF = allData.iloc[:,colMean]
colPlus=7*(i-1)+3
RF_plus1sigma = allData.iloc[:,colPlus]

etaIMc = np.interp(0.50,RF,IM)
IMc16 = np.interp(ss.norm.cdf(-1.0),RF,IM,left=np.nan,right=np.nan)
IMc84 = np.interp(ss.norm.cdf(1.0),RF,IM,left=np.nan,right=np.nan)
betaIMc = 0.5*math.log(IMc84/IMc16)
if np.isnan(IMc16):
    print('16th percentile cannot be defined on the fragility curve!')
    betaIMc = math.log(IMc84/etaIMc)
if np.isnan(IMc84):
    print('84th percentile cannot be defined on the fragility curve!')
    betaIMc = math.log(etaIMc/IMc16)

IM16 = np.interp(0.5,RF_plus1sigma,IM)
IM84 = np.interp(0.5,RF_minus1sigma,IM)
betaUF = 0.5*math.log(IM84/IM16)

(4) Having calculated the parameters of the fragility curve, they can be shown graphically on the associated fragility curve. Here, we show the median (here the 50th percentile IM), the 16th and 84th percentile IM's (eg, flow depth) on the related fragility curve. Recall that the logarithmic standard deviation is calculated as half of (natural log of) the ratio between the 84th and 16th percentile IM's.

In [ ]:
# Show the parameters of the fragility curves
fig.add_scatter(x=[0,etaIMc],y=[0.5,0.5],mode='lines',line=dict(color='rgb(82,82,82)',width=1.5,dash='dash'),showlegend=False)
fig.add_scatter(x=[etaIMc,etaIMc],y=[0,0.5],mode='lines',line=dict(color='rgb(82,82,82)',width=1.5,dash='dash'),showlegend=False)
fig.add_scatter(x=[etaIMc],y=[0.5],mode='markers',
                    marker=dict(size=8,color='rgb(82,82,82)',
                                line=dict(color='red',width=1)),name='median, p=50%')

fig.add_scatter(x=[0,IMc16],y=[ss.norm.cdf(-1.0),ss.norm.cdf(-1.0)],mode='lines',line=dict(color='rgb(82,82,82)',width=2,dash='dot'),showlegend=False)
fig.add_scatter(x=[IMc16,IMc16],y=[0,ss.norm.cdf(-1.0)],mode='lines',line=dict(color='rgb(82,82,82)',width=2,dash='dot'),showlegend=False)
fig.add_scatter(x=[IMc16],y=[ss.norm.cdf(-1.0)],mode='markers',
                    marker=dict(size=15,color='rgb(82,82,82)',symbol="star-triangle-up-dot",
                                line=dict(color='red',width=1)),name='16<sup>th</sup> percentile, p=16%')

fig.add_scatter(x=[0,IMc84],y=[ss.norm.cdf(1.0),ss.norm.cdf(1.0)],mode='lines',line=dict(color='rgb(82,82,82)',width=2,dash='dot'),showlegend=False)
fig.add_scatter(x=[IMc84,IMc84],y=[0,ss.norm.cdf(1.0)],mode='lines',line=dict(color='rgb(82,82,82)',width=2,dash='dot'),showlegend=False)
fig.add_scatter(x=[IMc84],y=[ss.norm.cdf(1.0)],mode='markers',
                    marker=dict(size=15,color='rgb(82,82,82)',symbol="star-triangle-down-dot",
                                line=dict(color='red',width=1)),name='84<sup>th</sup> percentile, p=84%')

fig.add_annotation(text='\u03B7<sub>IM<sub>C</sub></sub>', font=dict(color='black',size=16),ax=40,ay=-60,
                           x=etaIMc,y=0.50,
                           arrowhead=2,arrowsize=2,arrowwidth=1,arrowside='end',
                           align='left')

if np.isfinite(IMc84):
    fig.add_annotation(text='IM<sub>C</sup><sup>84</sup>', font=dict(color='black',size=16),ax=0,ay=-60,
                           x=IMc84,y=ss.norm.cdf(1.0),
                           arrowhead=2,arrowsize=2,arrowwidth=1,arrowside='end',
                           align='left')

if np.isfinite(IMc16):
    fig.add_annotation(text='IM<sub>C</sup><sup>16</sup>', font=dict(color='black',size=16),ax=60,ay=-60,
                           x=IMc16,y=ss.norm.cdf(-1.0),
                           arrowhead=2,arrowsize=2,arrowwidth=1,arrowside='end',
                           align='left')

fig.update_layout(title =dict(text='Fragility parameters',x=0.075,font=dict(size=30)),)

fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)

fig.show()



(5) Estimate the epistemic uncertainty: In this step, the epistemic uncertainty paramater can be estimated for the fragility curve corresponding to the selected damage level.This information can be interpreted as the uncertainty in the fragility median expressed as logarithmic standard deviation. This is roughly equal to half of (natural log of) the ratio of the median of the plus-one-sigma fragility curve to the median of the minus-one-sigma fragility curve. 

In [ ]:
# Estimating the parameters used for calculating the epistemic uncertainties

IM16 = np.interp(0.5,RF_plus1sigma,IM)
IM84 = np.interp(0.5,RF_minus1sigma,IM)
betaUF = 0.5*math.log(IM84/IM16)

(6) This value is visulaized below as the width of the confidence band (already plotted) at median:

In [ ]:
# Show the epistemic uncertainties

fig.add_scatter(x=[IM16,IM16],y=[0,0.5],mode='lines',line=dict(color='black',width=0.5,dash='dot'),showlegend=False)
fig.add_scatter(x=[IM16],y=[0.5],mode='markers',
                marker=dict(size=8,color='darkorange',symbol="star",
                            line=dict(color='dimgrey',width=1)),name='IM<sup>16</sup>',showlegend=False)

fig.add_scatter(x=[IM84,IM84],y=[0,0.5],mode='lines',line=dict(color='black',width=0.5,dash='dot'),showlegend=False)
fig.add_scatter(x=[IM84],y=[0.5],mode='markers',
                marker=dict(size=8,color='darkorange',symbol="star",
                            line=dict(color='dimgrey',width=1)),name='IM<sup>84</sup>',showlegend=False)


fig.add_annotation(text='IM<sup>84</sup>', font=dict(color='black',size=15),ax=40,ay=40,
                       x=IM84,y=0.50,
                       arrowhead=2,arrowsize=2,arrowwidth=1,arrowside='end',
                       align='right')

fig.add_annotation(text='IM<sup>16</sup>', font=dict(color='black',size=15),ax=-40,ay=40,
                       x=IM16,y=0.50,
                       arrowhead=2,arrowsize=2,arrowwidth=1,arrowside='end',
                       align='right')

fig.update_layout(title =dict(text='Epistemic uncertainty in the median of the fragility',x=0.075,font=dict(size=30)),)

fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)

fig.show()

Below, we can see a print of the fragility curve median (in meters), the logarithmic standard deviation (without units) and an estimate of the epistemic uncertainty in the fragility curve (without units).

In [ ]:
print('Fragility statistics for damage level = ', i)
print('median =', etaIMc, ' meters')
print('logarithmic standard deviation =', betaIMc)
print('epistemic uncertainty =', betaUF)